In [1]:
import pandas as pd
train_df=pd.read_csv("/content/augmented-sequential-masking-ner-mahasent.csv")
valid_df=pd.read_csv("/content/tweets-valid.csv")
test_df=pd.read_csv("/content/tweets-test.csv")

In [2]:
train_df.head()

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...,-1
1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...,-1
2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...,-1
3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...,1
4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1


In [3]:
valid_df.head()

,tweet,label
0,कोरोना काळात आपले पंतप्रधान मा. नरेंद्र मोदी...,1
1,पुणे येथे राष्ट्रसेवा दल अभ्यासिका विभाग यां...,1
2,"पुणे येथील ""क्रांती ज्योती"" या कार्यक्रमात ज...",0
3,आद्यक्रांतिकारक उमाजीराजे नाईक यांना जयंतीनि...,1
4,महाराष्ट्राचे मुख्यमंत्री म्हणून शपथ घेतल्या...,1


In [4]:
test_df.head()

,tweet,label
0,#BetiBachao चा फक्त नारा देऊन उपयोग नाही. महि...,1
1,पेट्रोल आणि डिझेलवर एक रूपया अधिभार लावल्यान...,-1
2,"लूट झूट का राज भागावो, \nरोजगार और विकास ला...",1
3,महाराष्ट्र विकास आघाडी सरकारच्या मंत्रिमंडळ ...,1
4,पिंपरीत अ‍ॅथलेटिक्स आणि विविध क्षेत्रांतल्या...,1


In [5]:
len(train_df)

11090

In [6]:
train_df['label'] = train_df['label'].replace(1,2)
test_df['label'] = test_df['label'].replace(1,2)
valid_df['label'] = valid_df['label'].replace(1,2)

train_df['label'] = train_df['label'].replace(0,1)
test_df['label'] = test_df['label'].replace(0,1)
valid_df['label'] = valid_df['label'].replace(0,1)

train_df['label'] = train_df['label'].replace(-1,0)
test_df['label'] = test_df['label'].replace(-1,0)
valid_df['label'] = valid_df['label'].replace(-1,0)

In [7]:
# data preprocessing:
# removing non marathi text 
import re
def remove_non_marathi_in_tweets(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [8]:
for i in range(len(train_df)):
  train_df['tweet'][i] = remove_non_marathi_in_tweets(train_df['tweet'][i])
for i in range(len(valid_df)):
  valid_df['tweet'][i] = remove_non_marathi_in_tweets(valid_df['tweet'][i])
for i in range(len(test_df)):
  test_df['tweet'][i] = remove_non_marathi_in_tweets(test_df['tweet'][i])

<ipython-input-8-482938b9e88c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['tweet'][i] = remove_non_marathi_in_tweets(train_df['tweet'][i])
<ipython-input-8-482938b9e88c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_df['tweet'][i] = remove_non_marathi_in_tweets(valid_df['tweet'][i])
<ipython-input-8-482938b9e88c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['tweet'][i] = remove_non_marathi_in_tweets(

In [9]:
train_df = train_df[train_df['tweet']!='']
valid_df = valid_df[valid_df['tweet']!='']
test_df = test_df[test_df['tweet']!='']

In [10]:
print(len(train_df))
print(len(valid_df))
print(len(test_df))

11087
1500
2250


In [11]:
train_df.drop_duplicates(inplace=True)
valid_df.drop_duplicates(inplace=True)
test_df.drop_duplicates(inplace=True)
train_df.dropna(inplace=True)
valid_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [12]:
print(len(train_df))
print(len(valid_df))
print(len(test_df))

11086
1500
2248


In [13]:
# shuffle the rows and reset the indices
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.sample(frac=1, random_state=42).reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [14]:
train_df

,tweet,label
0,सामाजिक न्याय विभागाच्या संजय गांधी निराधार यो...,2
1,कामगारांनी आपल्या गावी पायी चालत जाण्याचा धोका...,2
2,महाराष्ट्राचा इतिहास पाहता यातील सगळ्यात महत्त...,2
3,मराठवाडा खानदेश कोकण विदर्भात या सरकारमध्ये का...,2
4,आगामी निवडणूक राज्याची आहे आम्ही समविचारी पक्ष...,0
...,...,...
11081,गोरेगावमध्ये ३ वर्षांचा मुलगा गटारात पडून बेपत...,0
11082,महाराष्ट्र भाजपा कार्यकारिणीच्या नवनिर्वाचित प...,1
11083,डॉ आंबेडकरांनी कामगार कायदे केले लोकांना पाण्य...,2
11084,आज संध्याकाळी राज्याचे माननीय मुख्यमंत्री देवे...,1


In [15]:
train_df.to_csv('/content/data/tweets-train-new.csv')
test_df.to_csv('/content/data/tweets-test-new.csv')
valid_df.to_csv('/content/data/tweets-valid-new.csv')

In [16]:
!pip install --no-cache-dir transformers sentencepiece -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 175.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 316.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 381.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 367.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 184.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 226.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 262.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 379.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 302.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 385.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 369.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 306.7 MB/s eta 0:00:00


In [17]:
from datasets import load_dataset
data_files={"train":"/content/data/tweets-train-new.csv","test":"/content/data/tweets-test-new.csv","validation":"/content/data/tweets-valid-new.csv"}
dataset = load_dataset('/content/',data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/content-efc8568d824bf118/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

<ipython-input-18-4ff7ed1e6f07>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [19]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-bert-v2")


In [20]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=1)

In [23]:
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/11086 [00:00<?, ? examples/s]

Map:   0%|          | 0/2248 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [24]:
from transformers import AutoModelForSequenceClassification

In [25]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/marathi-bert-v2", num_labels=3)

Some weights of the model checkpoint at l3cube-pune/marathi-bert-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marat

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

In [27]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.486300,0.410361,0.859333


TrainOutput(global_step=1386, training_loss=0.5281041463216146, metrics={'train_runtime': 1137.1027, 'train_samples_per_second': 9.749, 'train_steps_per_second': 1.219, 'total_flos': 2916875348932608.0, 'train_loss': 0.5281041463216146, 'epoch': 1.0})

In [28]:
predictions = trainer.predict(tokenized_datasets["test"])

In [29]:
preds = np.argmax(predictions.predictions, axis=-1)

In [30]:
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8358540925266904}

In [31]:
def remove_non_marathi_goemotions(text):
    i= re.sub(r"http\S+", "", text)
    i= re.sub(r"pic\S+", "", i)
    i= re.sub(r"#", "", i)
    i= re.sub(r"lokmat\S+", "", i)
    i= re.sub(r"@\S+", "", i)
    i= re.sub(r"fb\S+", "", i)
    i= re.sub(r"twitter\S+", "", i)
    i= re.sub(r"bit\S+", "", i)    
    # i= re.sub(r"['\"!-./,;\[\]?]", "", i)  # during augmenting, keep the necessary punctuations [.?!'"] in the sentence.  
    i= re.sub(r"[\[\]]", "", i)    
    i= re.sub(r'[a-zA-Z0-9]', "", i)    
    i=re.split(r'[\s\n]+', i.strip())
    return(' '.join(i))

In [32]:
# prediction on GoEmotions test set:
test_df_goemotions=pd.read_csv("goemotions_subset_test_new.csv")
test_df_goemotions.drop(['Unnamed: 0.1', 'Unnamed: 0', 'text'], inplace=True, axis=1)
dict1 = {'marathi_text': 'tweet', 'three_point': 'label'}
test_df_goemotions.rename(columns=dict1, inplace=True)
test_df_goemotions = test_df_goemotions.iloc[:, [1, 0]]
test_df_goemotions['label'] = test_df_goemotions['label'].replace(1,2)
test_df_goemotions['label'] = test_df_goemotions['label'].replace(0,1)
test_df_goemotions['label'] = test_df_goemotions['label'].replace(-1,0)
for i in range(len(test_df_goemotions)):
  # test_df_goemotions['tweet'][i] = test_df_goemotions['tweet'][i].strip()
  test_df_goemotions['tweet'][i] = remove_non_marathi_goemotions(test_df_goemotions['tweet'][i])
test_df_goemotions = test_df_goemotions[test_df_goemotions['tweet']!='']
test_df_goemotions.drop_duplicates(inplace=True)
test_df_goemotions.dropna(inplace=True)
test_df_goemotions.to_csv('data/tweets-test-goemotions.csv')
data_files_goemotions={"test":"data/tweets-test-goemotions.csv"}
dataset_goemotions = load_dataset('/content/',data_files=data_files_goemotions)
tokenized_datasets_goemotions = dataset_goemotions.map(tokenize_function, batched=True)

<ipython-input-32-6506e949c4eb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_goemotions['tweet'][i] = remove_non_marathi_goemotions(test_df_goemotions['tweet'][i])
<ipython-input-32-6506e949c4eb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_goemotions.drop_duplicates(inplace=True)
<ipython-input-32-6506e949c4eb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_goemotions.dropna(inplace=True)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/content-d1f5ab3451c0c4e9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/8995 [00:00<?, ? examples/s]

In [33]:
predictions = trainer.predict(tokenized_datasets_goemotions["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.5371873262923846}